# Funwave-tvd 

## Time Series 1D

In [1]:
from ipywidgets import Box, Label, Text, Layout, Button, Output
from IPython.display import display, clear_output
oneDOutput = Output()
xTxt = Text()
yTxt = Text()
oneDBtn = Button(description='Show plot', icon='check')
oneDBox = Box([Label(value = 'Choose location (MAX[200][600]):'), xTxt, yTxt, oneDBtn], layout = Layout(width = '100%', justify_content = 'space-between') )

def oneDBtn_clicked(a):
    oneDOutput.clear_output()
    with oneDOutput:
        OneD(int(xTxt.value),int(yTxt.value))
        
oneDBtn.on_click(oneDBtn_clicked)

display(oneDBox)
display(oneDOutput)

opdir = "output/output/"
def OneD(a,b):
    plt.clf()
    plt.cla()
    plt.close()
    
    fig, ax = plt.subplots()
    
    X = []
    Y = []
    
    for frame in range(151):
        y = np.loadtxt(opdir+'eta_%05d' % (frame+1))
        Y.append(y[a][b])
        
    for i in range(151):
        time = '%.2f' % ((i)*0.2)
        X.append(time)

    unit ='(m)'
    ax.plot(X,Y)
    ax.set_title("At location ("+str(a)+")("+str(b)+")")
    ax.set_ylabel("eta "+unit)
    ax.set_xlabel("Time (s)")

    
    plt.show()
    

Box(children=(Label(value='Choose location (MAX[200][600]):'), Text(value=''), Text(value=''), Button(descript…

Output()

## Surface Elevation snapshot 2D

In [2]:
from ipywidgets import Box, Label, Text, Layout, Button, Output
from fwPlots import *
from IPython.display import display, clear_output
sesOutput = Output()
fileTxt = Text()
sesBtn = Button(description='Show plot', icon='check')
sesBox = Box([Label(value = 'Choose time from 0.2 to 30 (s): '), fileTxt, sesBtn],layout = Layout(width = '65%', justify_content = 'space-between'))


def sesBtn_clicked(a):
    sesOutput.clear_output()
    with sesOutput:
        twoDsnapPlot(int(float(fileTxt.value)/0.2))
    
sesBtn.on_click(sesBtn_clicked)

display(sesBox)
display(sesOutput)



Box(children=(Label(value='Choose time from 0.2 to 30 (s): '), Text(value=''), Button(description='Show plot',…

Output()

## Surface Elevation animation 2D

In [3]:
from ipywidgets import Box, Label, Text, Layout, Button, Output,FloatRangeSlider
from fwPlots import *
from IPython.display import display, clear_output
seaOut = Output()

seaFrs = FloatRangeSlider(value=[5,7], min=0.0, max=30, step=0.2,
                                 readout=True,readout_format='.2f', layout = Layout(width ="60%"))
seaBtn = Button(description='Show plot', icon='check')

seaBox = Box([Label(value = 'Choose time period:'),seaFrs, seaBtn], layout = Layout(width = '80%', justify_content = 'space-between'))

def seaBtn_clicked(a):
    anim = twoDsnapAnim(seaFrs.value[0],seaFrs.value[1], 0.2)
    seaOut.clear_output()
    with seaOut:
        display(HTML(anim.to_html5_video()))
seaBtn.on_click(seaBtn_clicked)

display(seaBox)
display(seaOut)



Box(children=(Label(value='Choose time period:'), FloatRangeSlider(value=(5.0, 7.0), layout=Layout(width='60%'…

Output()

## Cross-shore Profile animation 2D

In [4]:
from ipywidgets import Box, Label, Text, Layout, Button, Output ,FloatRangeSlider, VBox
from fwPlots import *
from IPython.display import display, clear_output
cpaOut = Output()

depthTxt = Text()

depthBox = Box([Label(value='Choose depth from 1 to 200:'), depthTxt],layout = Layout(width = '50%', justify_content = 'space-between'))

cpaFrs = FloatRangeSlider(value=[5,7], min=0.0, max=30, step=0.2,
                                 readout=True,readout_format='.2f', layout = Layout(width ="60%"))
cpaBtn = Button(description='Show plot', icon='check')

sldBox = Box([Label(value = 'Choose time period:'),cpaFrs, cpaBtn], layout = Layout(width = '80%', justify_content = 'space-between'))

cpaBox = VBox([depthBox, sldBox])

def cpaBtn_clicked(a):
    anim = depProfileWithEta(depthTxt.value, cpaFrs.value[0], cpaFrs.value[1], 30.0, 0.2, 600)
    cpaOut.clear_output()
    with cpaOut:
        display(HTML(anim.to_html5_video()))
cpaBtn.on_click(cpaBtn_clicked)


display(cpaBox)
display(cpaOut)

Output()